<a href="https://colab.research.google.com/github/GraceW18/image-classification-model-CNN/blob/main/RandomizedSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
import random
import time

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Load and preprocess data
train_data = pd.read_csv('sign_mnist_13bal_train.csv')
test_data = pd.read_csv('sign_mnist_13bal_test.csv')

X_train = train_data.drop('class', axis=1).values / 255.0
y_train = train_data['class'].values
X_test = test_data.drop('class', axis=1).values / 255.0
y_test = test_data['class'].values

y_train[y_train == 10] = 9
y_test[y_test == 10] = 9

print("Unique labels:", np.unique(y_train))

# Reshape as images for CNN input (28x28 grayscale)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
num_classes = len(np.unique(y_train))

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Number of classes: {num_classes}")

# Define which parameters are for model building vs training
MODEL_PARAMS = ['filters', 'dropout_rate', 'dense_units', 'learning_rate', 'optimizer']
TRAINING_PARAMS = ['epochs', 'batch_size']

def build_and_compile_model(filters=32, dropout_rate=0.3, dense_units=64,
                           learning_rate=0.001, optimizer='adam'):
    """
    Build and compile CNN model - ONLY accepts model architecture parameters
    """
    print(f"    Building model: filters={filters}, dropout={dropout_rate}, units={dense_units}, lr={learning_rate}, opt={optimizer}")

    model = Sequential([
        tf.keras.Input(shape=(28, 28, 1)),
        Conv2D(filters, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(filters * 2, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])

    # Configure optimizer
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate, momentum=0.9)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate)

    model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def evaluate_model_cv(param_dict, X_train, y_train, cv_folds=3):
    """
    Evaluate model using cross-validation
    """
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_scores = []

    # Extract model parameters (only the ones the function accepts)
    model_params = {
        'filters': param_dict['filters'],
        'dropout_rate': param_dict['dropout_rate'],
        'dense_units': param_dict['dense_units'],
        'learning_rate': param_dict['learning_rate'],
        'optimizer': param_dict['optimizer']
    }

    # Extract training parameters
    epochs = param_dict['epochs']
    batch_size = param_dict['batch_size']

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        print(f"    Fold {fold + 1}/{cv_folds}", end=" ")

        # Split data
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Build model with ONLY model parameters
        model = build_and_compile_model(**model_params)

        # Train model with training parameters
        history = model.fit(
            X_train_fold, y_train_fold,
            epochs=epochs,
            batch_size=min(batch_size, len(X_train_fold)),
            validation_data=(X_val_fold, y_val_fold),
            verbose=0
        )

        # Get best validation accuracy
        best_val_acc = max(history.history['val_accuracy'])
        cv_scores.append(best_val_acc)
        print(f"Score: {best_val_acc:.3f}")

        # Clear memory
        del model
        tf.keras.backend.clear_session()

    return np.mean(cv_scores), np.std(cv_scores)

# Define parameter ranges optimized for small dataset
param_ranges = {
    'filters': [16, 24, 32],
    'dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'dense_units': [32, 64, 128],
    'learning_rate': [0.001, 0.005, 0.01],
    'optimizer': ['adam', 'rmsprop'],
    'epochs': [30, 50, 80],
    'batch_size': [8, 16, 32]
}

def generate_random_params(param_ranges, n_combinations=12):
    """
    Generate random parameter combinations
    """
    combinations = []

    for _ in range(n_combinations):
        params = {}
        for param, values in param_ranges.items():
            params[param] = random.choice(values)
        combinations.append(params)

    return combinations

# Perform randomized search
print("\nGenerating random parameter combinations...")
param_combinations = generate_random_params(param_ranges, n_combinations=12)

print("Starting Randomized Parameter Search...")
print("="*60)

results = []
best_score = 0
best_params = None

for i, params in enumerate(param_combinations):
    print(f"\nCombination {i+1}/{len(param_combinations)}")
    print("-" * 50)

    # Print current parameters
    print("Parameters:")
    for param, value in params.items():
        print(f"  {param}: {value}")

    start_time = time.time()

    try:
        # Evaluate model
        mean_score, std_score = evaluate_model_cv(params, X_train, y_train, cv_folds=3)

        elapsed_time = time.time() - start_time

        print(f"  CV Result: {mean_score:.4f} (+/- {std_score*2:.4f})")
        print(f"  Time: {elapsed_time:.1f}s")

        # Store results
        results.append({
            'params': params.copy(),
            'mean_score': mean_score,
            'std_score': std_score,
            'time': elapsed_time
        })

        # Update best parameters
        if mean_score > best_score:
            best_score = mean_score
            best_params = params.copy()
            print("  *** NEW BEST SCORE! ***")

    except Exception as e:
        print(f"  ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

# Print final results
print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)

if best_params is not None:
    print(f"Best CV score: {best_score:.4f}")
    print("Best parameters:")
    for param, value in best_params.items():
        print(f"  {param}: {value}")

    # Train final model
    print("\n" + "-"*40)
    print("Training final model on full training set...")

    # Extract model parameters for final model
    final_model_params = {
        'filters': best_params['filters'],
        'dropout_rate': best_params['dropout_rate'],
        'dense_units': best_params['dense_units'],
        'learning_rate': best_params['learning_rate'],
        'optimizer': best_params['optimizer']
    }

    final_model = build_and_compile_model(**final_model_params)

    # Train with best parameters
    history = final_model.fit(
        X_train, y_train,
        epochs=best_params['epochs'],
        batch_size=min(best_params['batch_size'], len(X_train)),
        validation_split=0.15,
        verbose=1
    )

    # Evaluate on test set
    print("\n" + "-"*40)
    print("Test set evaluation:")

    test_predictions = final_model.predict(X_test, verbose=0)
    test_pred_classes = np.argmax(test_predictions, axis=1)
    test_accuracy = accuracy_score(y_test, test_pred_classes)

    print(f"Test accuracy: {test_accuracy:.4f}")
    print(f"Test error: {1 - test_accuracy:.4f}")

    # Show classification report
    print("\nClassification Report:")
    print(classification_report(y_test, test_pred_classes))

    # Show top results
    print("\n" + "-"*40)
    print("Top 5 parameter combinations:")

    if results:
        results_sorted = sorted(results, key=lambda x: x['mean_score'], reverse=True)

        for i, result in enumerate(results_sorted[:5]):
            print(f"\n{i+1}. CV Score: {result['mean_score']:.4f} (+/- {result['std_score']*2:.4f})")
            print(f"   Time: {result['time']:.1f}s")
            print("   Parameters:")
            for param, value in result['params'].items():
                print(f"     {param}: {value}")

    # Save results
    try:
        final_model.save('best_small_dataset_model.h5')

        with open('best_parameters_small.txt', 'w') as f:
            f.write("Best Parameters for Small Dataset:\n")
            f.write("="*50 + "\n")
            for param, value in best_params.items():
                f.write(f"{param}: {value}\n")
            f.write(f"\nCV Score: {best_score:.4f}\n")
            f.write(f"Test Accuracy: {test_accuracy:.4f}\n")
            f.write(f"Dataset size: {len(X_train)} training samples\n")

        print(f"\nModel saved as 'best_small_dataset_model.h5'")
        print("Parameters saved to 'best_parameters_small.txt'")

    except Exception as e:
        print(f"Couldn't save files: {str(e)}")

else:
    print("No successful parameter combinations found!")
    print("Check your data and try reducing parameter ranges.")

print("\nParameter search completed!")

Unique labels: [0 1 2 3 4 5 6 7 8 9]
Training data shape: (130, 28, 28, 1)
Test data shape: (130, 28, 28, 1)
Number of classes: 10

Generating random parameter combinations...
Starting Randomized Parameter Search...

Combination 1/12
--------------------------------------------------
Parameters:
  filters: 32
  dropout_rate: 0.2
  dense_units: 32
  learning_rate: 0.01
  optimizer: rmsprop
  epochs: 30
  batch_size: 8
    Fold 1/3     Building model: filters=32, dropout=0.2, units=32, lr=0.01, opt=rmsprop
Score: 0.091
    Fold 2/3     Building model: filters=32, dropout=0.2, units=32, lr=0.01, opt=rmsprop
Score: 0.814
    Fold 3/3     Building model: filters=32, dropout=0.2, units=32, lr=0.01, opt=rmsprop
Score: 0.116
  CV Result: 0.3404 (+/- 0.6701)
  Time: 32.6s
  *** NEW BEST SCORE! ***

Combination 2/12
--------------------------------------------------
Parameters:
  filters: 16
  dropout_rate: 0.2
  dense_units: 128
  learning_rate: 0.01
  optimizer: adam
  epochs: 80
  batch_size:

Test accuracy: 0.7923
Test error: 0.2077

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.54      0.64        13
           1       0.91      0.77      0.83        13
           2       1.00      0.92      0.96        13
           3       0.77      0.77      0.77        13
           4       0.71      0.77      0.74        13
           5       0.87      1.00      0.93        13
           6       0.60      0.69      0.64        13
           7       0.80      0.62      0.70        13
           8       0.87      1.00      0.93        13
           9       0.69      0.85      0.76        13

    accuracy                           0.79       130
   macro avg       0.80      0.79      0.79       130
weighted avg       0.80      0.79      0.79       130


----------------------------------------
Top 5 parameter combinations:

1. CV Score: 0.9077 (+/- 0.0760)
   Time: 69.1s
   Parameters:
     filters: 24
     dropout_rate: 0.4
